In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

# Load Test file

In [2]:
# load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
# file_format = '{}_features_supplementV1_countALL.csv'
# test_all = pd.read_csv(load_path+file_format.format('testsupplement'))
# test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
# test = test_all.iloc[test_index]
# del test_all
# del test_index
# gc.collect()

In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_equalhour_supplementV1.csv'
test = pd.read_csv(load_path+file_format.format('test'))

# Train on Day 9, Validation on Day 8

In [4]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_supplementV1_countALL.csv'

trainday = 'day9'
valday = 'day8'
trainset = pd.read_csv(load_path+file_format.format(trainday))
print('load trainset done!')
valset = pd.read_csv(load_path+file_format.format(valday))
print('load valset done!')

load trainset done!
load valset done!


# define parameter

In [5]:
target = 'is_attributed'

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':60,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(test.columns.values)

In [6]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
# extra = set(['ip_device_os_countself',
#              'ip_day_hour_countself', 
#              'app_day_hour_countself',
#              'ip_app_device_os_countself',
#              'ip_app_day_hour_countself', 
#              'ip_os_day_hour_countself', 
#              'ip_app_os_day_hour_countself' ])

# extra = set([])
extra = set(['ip_device_os_count',
             'ip_day_hour_count', 
             'app_day_hour_count',
             'ip_app_device_os_count',
             'ip_app_day_hour_count', 
             'ip_os_day_hour_count', 
             'ip_app_os_day_hour_count' ])
feature_cols = list(set(feature_cols) - extra)

In [7]:
feature_cols

['matrixFact_user_ipchannel_item_appdeviceos',
 'ip_app_device_os_regression',
 'ip_app_device_regression',
 'ip_app_device_lasttimediff',
 'ip_app_day_hour_countself',
 'ip_app_device_time2previousclick',
 'ip_device_os_countfromfuture',
 'ip_device_os_time2nextclick',
 'ip_app_os_day_hour_countself',
 'ip_device_os_firsttimediff',
 'ip_app_device_os_countfrompast',
 'ip_app_device_countfrompast',
 'channel',
 'ip_app_device_os_countself',
 'ip_day_hour_countself',
 'app_day_hour_countself',
 'ip_app_device_os_mean',
 'app_device_os_mean',
 'ip_app_device_os_channel_regression',
 'ip_app_device_os_firsttimediff',
 'app',
 'ip_device_os_countself',
 'ip_app_device_mean',
 'ip_device_os_countfrompast',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_firsttimediff',
 'ip_os_day_hour_countself',
 'ip_app_device_countfromfuture',
 'ip_app_device_os_lasttimediff',
 'matrixFact_user_iposdeviceapp_item_app',
 'ip_app_device_os_countfromfuture',
 'os',
 'ip_app_device_os_time2previousclick

In [8]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [9]:
file_name = 'train_{}_val{}_supplement_countonSELF.npy'.format(trainday, valday)
model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
best_round = model.best_iteration
# df_all = pd.concat([trainset, valset])
# model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
# del df_all
gc.collect()
pred = model.predict(test[feature_cols])
np.save(load_path+file_name, pred)

# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_{}_val{}_supplement_countonSELF.csv.gz'.format(trainday,valday), compression='gzip', index=False)

    

start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 60 rounds.
[10]	valid_0's auc: 0.98032
[20]	valid_0's auc: 0.982171
[30]	valid_0's auc: 0.983535
[40]	valid_0's auc: 0.984223
[50]	valid_0's auc: 0.984707
[60]	valid_0's auc: 0.984992
[70]	valid_0's auc: 0.98516
[80]	valid_0's auc: 0.985244
[90]	valid_0's auc: 0.985288
[100]	valid_0's auc: 0.985335
[110]	valid_0's auc: 0.985359
[120]	valid_0's auc: 0.985374
[130]	valid_0's auc: 0.985378
[140]	valid_0's auc: 0.985386
[150]	valid_0's auc: 0.985394
[160]	valid_0's auc: 0.985367
[170]	valid_0's auc: 0.985361
[180]	valid_0's auc: 0.98533
[190]	valid_0's auc: 0.985327
[200]	valid_0's auc: 0.985311
Early stopping, best iteration is:
[148]	valid_0's auc: 0.985404
loading file done!
predicting file done!


In [10]:
importance = pd.Series(model.feature_importance(importance_type='gain'), index=feature_cols)
# importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

matrixFact_user_ip_item_appdeviceos           1.983543e+08
matrixFact_user_iposdeviceapp_item_app        1.286551e+08
matrixFact_user_ipchannel_item_appdeviceos    5.674032e+07
app_device_os_mean                            2.989845e+07
ip_app_device_os_time2nextclick               2.038656e+07
channel                                       1.844391e+07
ip_day_hour_countself                         9.260188e+06
app                                           9.146928e+06
os                                            9.127298e+06
hour                                          5.422664e+06
ip_app_device_os_lasttimediff                 5.092837e+06
ip_app_day_hour_countself                     2.632952e+06
device                                        2.565647e+06
ip_device_os_countself                        2.278079e+06
ip_app_device_time2nextclick                  2.175525e+06
ip_app_device_mean                            2.082411e+06
ip_app_os_day_hour_countself                  1.762402e+

# Average

In [25]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixregV5_leaf61_firsttimediff_timeinTrainTest.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixregV5_leaf61_firsttimediff_timeinTrainTest.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixregV5_leaf61_firsttimediff_timeinTrainTest.npy')

pred = (day7 + day8 + day9)/3

In [26]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixregV5_leaf61_firsttimediff_timeinTrainTest.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
